# Importy

In [70]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import re
from collections import Counter
import ast

# Podstawowe wczytanie

In [100]:
# podstawowe wczytanie plików
listings = pd.read_csv('listings.csv', sep=';')
calendar = pd.read_csv('calendar.csv', sep=';')

# usunięcie $ i konwersja price na float
calendar['price'] = calendar['price'].str.replace(r'[\$,]', '', regex=True).astype(float)

In [101]:
# zostawiamy tylko wybrane kolumny
columns_to_keep = [
    'id',
    'neighbourhood_cleansed',
    'neighbourhood_group_cleansed',
    'latitude',
    'longitude',
    'property_type',
    'room_type',
    'accommodates',
    'bathrooms',
    'bathrooms_text',
    'bedrooms',
    'beds',
    'amenities',
    'instant_bookable',
    'price'
]

listings = listings[columns_to_keep]

In [102]:
print(listings.head())

            id           neighbourhood_cleansed neighbourhood_group_cleansed  \
0     16989407                la Vila de Gràcia                       Gràcia   
1     34133454                la Vila de Gràcia                       Gràcia   
2  1,04185E+18           la Dreta de l'Eixample                     Eixample   
3  6,95612E+17                      el Poblenou                   Sant Martí   
4       847195  l'Antiga Esquerra de l'Eixample                     Eixample   

    latitude  longitude                property_type        room_type  \
0  41.409920   2.157330         Private room in home     Private room   
1  41.397630   2.159340    Entire serviced apartment  Entire home/apt   
2  41.394798   2.165613           Entire rental unit  Entire home/apt   
3  41.399490   2.202610  Private room in rental unit     Private room   
4  41.385080   2.155270           Entire rental unit  Entire home/apt   

   accommodates  bathrooms    bathrooms_text  bedrooms  beds  \
0             2 

# Dodanie ceny (target)

In [103]:
# chcemy dla każdego listingu mieć jedną cenę (price z listings_excel_ready nie spełnia tego, bo a)to jest 'local currency', a b)są braki dla niektórych listingów
# można usunąć tą kolumnę
listings.drop(columns=['price'], inplace=True)
# więc sprawdzamy czy każdy listing z listings_excel_ready ma przynajmniej jeden wpis w calendar
listings_ids = set(listings['id'])
calendar_listing_ids = set(calendar['listing_id'])

missing_in_calendar = listings_ids - calendar_listing_ids

if missing_in_calendar:
    print(f"Brak wpisów w calendar dla listingów o ID: {missing_in_calendar}")
else:
    print("Każdy listing z listings_excel_ready ma wpis w calendar.")

Każdy listing z listings_excel_ready ma wpis w calendar.


In [104]:
# skoro tak to możemy tego użyć - chcemy połączyć listingi z odpowiednią ceną, sprawdzamy ile jest unikalnych wartości
price_stats = calendar.groupby('listing_id')['price'].agg(
    nunique='nunique',
    min='min',
    max='max',
    mean='mean',
    median='median'
).reset_index()

# niektóre listingi mają różne ceny w zależnosci od dnia - my chcemy jedną wartość, dlatego dla tych przypadków bierzemy medianę
price_stats['final_price'] = price_stats.apply(
    lambda row: row['median'] if row['nunique'] > 1 else row['min'],
    axis=1
)

In [105]:
# dodajemy kolumnę
listings = listings.merge(price_stats[['listing_id', 'final_price']], left_on='id', right_on='listing_id', how='left')

listings.drop(columns=['listing_id'], inplace=True)
listings.rename(columns={'final_price': 'price'}, inplace=True)

# Uzupełnienie brakujących danych

In [106]:
# w tych kolumnach dane są niepełne
columns_with_nan = listings.columns[listings.isnull().any()].tolist()
print('Kolumny, w których są puste wartości: ', columns_with_nan)

Kolumny, w których są puste wartości:  ['bathrooms', 'bathrooms_text', 'bedrooms', 'beds']


## Dodanie mediany do bedrooms i beds

In [107]:
# bedrooms i beds wypełniamy poprzez medianę z pozostałych listingów
listings['bedrooms'] = listings['bedrooms'].fillna(listings['bedrooms'].median())
listings['beds'] = listings['beds'].fillna(listings['beds'].median())

In [108]:
# zostały bathrooms i bathrooms_text
columns_with_nan = listings.columns[listings.isnull().any()].tolist()
print('Kolumny, w których są puste wartości: ', columns_with_nan)

Kolumny, w których są puste wartości:  ['bathrooms', 'bathrooms_text']


## Ujednolicenie bathrooms i bathrooms_text

In [109]:
def normalize_half_bath(value):
    if pd.isna(value):
        return value
    value = value.strip().lower()
    if value == 'half-bath':
        return '0.5 bath'
    elif value == 'shared half-bath':
        return '0.5 shared bath'
    elif value == 'private half-bath':
        return '0.5 private bath'
    return value

# pojedyncze wartości odbiegają od typowego 'liczba + suffix', więc naprawiamy to
listings['bathrooms_text'] = listings['bathrooms_text'].apply(normalize_half_bath)

In [110]:
def extract_bathrooms_num(text):
    if pd.isna(text) or text == 'nan':
        return np.nan
    match = re.match(r'^(\d*\.?\d+)', text)
    if match:
        return float(match.group(1))
    return np.nan

def extract_suffix(text):
    if pd.isna(text) or text == 'nan':
        return np.nan
    match = re.match(r'^\d*\.?\d+\s*(.*)$', text)
    if match:
        return match.group(1).strip()
    return np.nan

# dodajemy bathrooms_num i bathrooms_suffix - to będą nasze dane o łazienkach
listings['bathrooms_text_num'] = listings['bathrooms_text'].apply(extract_bathrooms_num)
listings['bathrooms_suffix'] = listings['bathrooms_text'].apply(extract_suffix)
listings['bathrooms_num_old'] = pd.to_numeric(listings['bathrooms'], errors='coerce')
listings['bathrooms_num'] = listings['bathrooms_text_num'].combine_first(listings['bathrooms_num_old'])
listings['bathrooms_suffix'] = listings['bathrooms_suffix'].fillna('missing')

In [111]:
print(listings.head())
print('Możliwe suffixy: ', listings['bathrooms_suffix'].unique())

            id           neighbourhood_cleansed neighbourhood_group_cleansed  \
0     16989407                la Vila de Gràcia                       Gràcia   
1     34133454                la Vila de Gràcia                       Gràcia   
2  1,04185E+18           la Dreta de l'Eixample                     Eixample   
3  6,95612E+17                      el Poblenou                   Sant Martí   
4       847195  l'Antiga Esquerra de l'Eixample                     Eixample   

    latitude  longitude                property_type        room_type  \
0  41.409920   2.157330         Private room in home     Private room   
1  41.397630   2.159340    Entire serviced apartment  Entire home/apt   
2  41.394798   2.165613           Entire rental unit  Entire home/apt   
3  41.399490   2.202610  Private room in rental unit     Private room   
4  41.385080   2.155270           Entire rental unit  Entire home/apt   

   accommodates  bathrooms    bathrooms_text  bedrooms  beds  \
0             2 

In [112]:
# usuwamy niepotrzebne stare kolumny
listings.drop(columns=['bathrooms', 'bathrooms_text', 'bathrooms_text_num', 'bathrooms_num_old'], inplace=True)

In [113]:
# zamiast pola suffix - chcemy mieć typ - shared, private albo nieznany
def classify_bathroom_type(suffix):
    suffix = suffix.lower()
    if 'shared' in suffix:
        return 'shared'
    elif 'private' in suffix:
        return 'private'
    else:
        return 'unknown'

listings['bathroom_type'] = listings['bathrooms_suffix'].apply(classify_bathroom_type)

In [114]:
# usuwamy kolumnę suffix - mamy liczbę łazienek i ich typ
listings.drop(columns=['bathrooms_suffix'], inplace=True)
print(listings.head())
print('Możliwe wartości type: ', listings['bathroom_type'].unique())

            id           neighbourhood_cleansed neighbourhood_group_cleansed  \
0     16989407                la Vila de Gràcia                       Gràcia   
1     34133454                la Vila de Gràcia                       Gràcia   
2  1,04185E+18           la Dreta de l'Eixample                     Eixample   
3  6,95612E+17                      el Poblenou                   Sant Martí   
4       847195  l'Antiga Esquerra de l'Eixample                     Eixample   

    latitude  longitude                property_type        room_type  \
0  41.409920   2.157330         Private room in home     Private room   
1  41.397630   2.159340    Entire serviced apartment  Entire home/apt   
2  41.394798   2.165613           Entire rental unit  Entire home/apt   
3  41.399490   2.202610  Private room in rental unit     Private room   
4  41.385080   2.155270           Entire rental unit  Entire home/apt   

   accommodates  bedrooms  beds  \
0             2       1.0   2.0   
1         

In [115]:
# sprawdzamy czy jakieś wiersze mają jeszcze brakujące wartości
rows_with_nan = listings[listings.isnull().any(axis=1)]

if not rows_with_nan.empty:
    print(f"Liczba wierszy z brakami: {len(rows_with_nan)}")
    for index, row in rows_with_nan.iterrows():
        missing_cols = row[row.isnull()].index.tolist()
        print(f"Listing ID: {row['id']}, Brakujące kolumny: {missing_cols}")
else:
    print("Brak brakujących danych (NaN) w listings.")

Liczba wierszy z brakami: 2
Listing ID: 9,39973E+17, Brakujące kolumny: ['bathrooms_num']
Listing ID: 1,1514E+18, Brakujące kolumny: ['bathrooms_num']


In [116]:
# te dwa wiersze nie miały ani pola bathrooms, ani bathrooms_text - uzupełniamy liczbę medianą tak jak w przypadku beds
listings['bathrooms_num'] = listings['bathrooms_num'].fillna(listings['bathrooms_num'].median())

In [117]:
# teraz już nie ma brakujących danych
rows_with_nan = listings[listings.isnull().any(axis=1)]

if not rows_with_nan.empty:
    print(f"Liczba wierszy z brakami: {len(rows_with_nan)}")
    for index, row in rows_with_nan.iterrows():
        missing_cols = row[row.isnull()].index.tolist()
        print(f"Listing ID: {row['id']}, Brakujące kolumny: {missing_cols}")
else:
    print("Brak brakujących danych (NaN) w listings.")

Brak brakujących danych (NaN) w listings.


In [118]:
print(listings)

               id           neighbourhood_cleansed  \
0        16989407                la Vila de Gràcia   
1        34133454                la Vila de Gràcia   
2     1,04185E+18           la Dreta de l'Eixample   
3     6,95612E+17                      el Poblenou   
4          847195  l'Antiga Esquerra de l'Eixample   
...           ...                              ...   
5821  6,94423E+17                            Horta   
5822      3905541            el Besòs i el Maresme   
5823     22101214               la Sagrada Família   
5824  1,21366E+18           la Dreta de l'Eixample   
5825  7,72119E+17           la Dreta de l'Eixample   

     neighbourhood_group_cleansed   latitude  longitude  \
0                          Gràcia  41.409920   2.157330   
1                          Gràcia  41.397630   2.159340   
2                        Eixample  41.394798   2.165613   
3                      Sant Martí  41.399490   2.202610   
4                        Eixample  41.385080   2.155270 

## Zamiana instant_bookable

In [119]:
# zamiast instant bookable t/f chcemy 1/0
listings['instant_bookable'] = listings['instant_bookable'].map({'t': 1, 'f': 0})

In [120]:
print(listings)

               id           neighbourhood_cleansed  \
0        16989407                la Vila de Gràcia   
1        34133454                la Vila de Gràcia   
2     1,04185E+18           la Dreta de l'Eixample   
3     6,95612E+17                      el Poblenou   
4          847195  l'Antiga Esquerra de l'Eixample   
...           ...                              ...   
5821  6,94423E+17                            Horta   
5822      3905541            el Besòs i el Maresme   
5823     22101214               la Sagrada Família   
5824  1,21366E+18           la Dreta de l'Eixample   
5825  7,72119E+17           la Dreta de l'Eixample   

     neighbourhood_group_cleansed   latitude  longitude  \
0                          Gràcia  41.409920   2.157330   
1                          Gràcia  41.397630   2.159340   
2                        Eixample  41.394798   2.165613   
3                      Sant Martí  41.399490   2.202610   
4                        Eixample  41.385080   2.155270 

# One-hot encoding

In [121]:
# listę udogodnień chcemy załatwić przez one-hot-encoding - wybieramy 50 najpopularniejszych
all_amenities = Counter(item
    for amenities in listings['amenities']
    for item in ast.literal_eval(amenities)
)

In [122]:
print(all_amenities.most_common(50))

[('Wifi', 5395), ('Kitchen', 5245), ('Hot water', 4332), ('Hair dryer', 4279), ('Hangers', 4194), ('Essentials', 4190), ('Iron', 4120), ('Dishes and silverware', 3948), ('Bed linens', 3789), ('Refrigerator', 3788), ('TV', 3653), ('Cooking basics', 3606), ('Heating', 3540), ('Washer', 3523), ('Microwave', 3522), ('Air conditioning', 3298), ('Elevator', 3065), ('Dedicated workspace', 2679), ('Shampoo', 2678), ('Oven', 2648), ('Coffee maker', 2552), ('Hot water kettle', 2312), ('Long term stays allowed', 2274), ('Freezer', 2240), ('Toaster', 2122), ('Dishwasher', 2100), ('Dining table', 2004), ('Stove', 1877), ('Cleaning products', 1807), ('Drying rack for clothing', 1793), ('Wine glasses', 1772), ('Host greets you', 1751), ('Room-darkening shades', 1714), ('Extra pillows and blankets', 1635), ('Shower gel', 1630), ('Body soap', 1526), ('Smoke alarm', 1512), ('Fire extinguisher', 1268), ('First aid kit', 1218), ('Luggage dropoff allowed', 1194), ('Self check-in', 1190), ('Patio or balcony

In [123]:
# zamieniamy listę na wartości 1/0 w odpowiednich utworzonych przez one-hot kolumnach
top_amenities = [item for item, count in all_amenities.most_common(50)]

def has_amenity(amenities_str, amenity):
    amenities_list = ast.literal_eval(amenities_str)
    return int(amenity in amenities_list)

for amenity in top_amenities:
    listings[f'amenity_{amenity}'] = listings['amenities'].apply(lambda x: has_amenity(x, amenity))

listings.drop(columns=['amenities'], inplace=True)

In [124]:
print(listings.columns)

Index(['id', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed',
       'latitude', 'longitude', 'property_type', 'room_type', 'accommodates',
       'bedrooms', 'beds', 'instant_bookable', 'price', 'bathrooms_num',
       'bathroom_type', 'amenity_Wifi', 'amenity_Kitchen', 'amenity_Hot water',
       'amenity_Hair dryer', 'amenity_Hangers', 'amenity_Essentials',
       'amenity_Iron', 'amenity_Dishes and silverware', 'amenity_Bed linens',
       'amenity_Refrigerator', 'amenity_TV', 'amenity_Cooking basics',
       'amenity_Heating', 'amenity_Washer', 'amenity_Microwave',
       'amenity_Air conditioning', 'amenity_Elevator',
       'amenity_Dedicated workspace', 'amenity_Shampoo', 'amenity_Oven',
       'amenity_Coffee maker', 'amenity_Hot water kettle',
       'amenity_Long term stays allowed', 'amenity_Freezer', 'amenity_Toaster',
       'amenity_Dishwasher', 'amenity_Dining table', 'amenity_Stove',
       'amenity_Cleaning products', 'amenity_Drying rack for clothing',
     

In [125]:
# w tych kolumnach stosujemy one-hot
categorical_cols = ['property_type', 'room_type', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'bathroom_type']

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_array = encoder.fit_transform(listings[categorical_cols])
encoded_cols = encoder.get_feature_names_out(categorical_cols)
encoded_df = pd.DataFrame(encoded_array, columns=encoded_cols, index=listings.index)
listings = pd.concat([listings.drop(columns=categorical_cols), encoded_df], axis=1)

# Zamiana kolejności

In [126]:
cols = [col for col in listings.columns if col != 'price'] + ['price']
listings = listings[cols]

# Finalny efekt - dane po obróbce

In [127]:
for col in listings.columns:
    print(f"{col}: {listings[col].dtype}")

id: object
latitude: float64
longitude: float64
accommodates: int64
bedrooms: float64
beds: float64
instant_bookable: int64
bathrooms_num: float64
amenity_Wifi: int64
amenity_Kitchen: int64
amenity_Hot water: int64
amenity_Hair dryer: int64
amenity_Hangers: int64
amenity_Essentials: int64
amenity_Iron: int64
amenity_Dishes and silverware: int64
amenity_Bed linens: int64
amenity_Refrigerator: int64
amenity_TV: int64
amenity_Cooking basics: int64
amenity_Heating: int64
amenity_Washer: int64
amenity_Microwave: int64
amenity_Air conditioning: int64
amenity_Elevator: int64
amenity_Dedicated workspace: int64
amenity_Shampoo: int64
amenity_Oven: int64
amenity_Coffee maker: int64
amenity_Hot water kettle: int64
amenity_Long term stays allowed: int64
amenity_Freezer: int64
amenity_Toaster: int64
amenity_Dishwasher: int64
amenity_Dining table: int64
amenity_Stove: int64
amenity_Cleaning products: int64
amenity_Drying rack for clothing: int64
amenity_Wine glasses: int64
amenity_Host greets you: i

In [128]:
print(listings)

               id   latitude  longitude  accommodates  bedrooms  beds  \
0        16989407  41.409920   2.157330             2       1.0   2.0   
1        34133454  41.397630   2.159340             3       1.0   1.0   
2     1,04185E+18  41.394798   2.165613             2       1.0   1.0   
3     6,95612E+17  41.399490   2.202610             2       1.0   1.0   
4          847195  41.385080   2.155270             4       2.0   3.0   
...           ...        ...        ...           ...       ...   ...   
5821  6,94423E+17  41.435440   2.159670             3       1.0   3.0   
5822      3905541  41.417370   2.217320             6       2.0   3.0   
5823     22101214  41.405109   2.175780             5       2.0   4.0   
5824  1,21366E+18  41.391790   2.171751             2       1.0   1.0   
5825  7,72119E+17  41.397334   2.165958             3       2.0   2.0   

      instant_bookable  bathrooms_num  amenity_Wifi  amenity_Kitchen  ...  \
0                    0            1.5         

In [129]:
listings.to_csv('final_data.csv', sep=';', index=False)